In [9]:
import requests 
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
from ast import literal_eval
from collections import Counter
import re

In [10]:
df = pd.read_csv('cardamom_recipes.csv')

,Unnamed: 0,recipe,link,ingredients,error
85,85,Garam Masala,https://cooking.nytimes.com/recipes/1021923-ga...,"['Cardamom', 'Cardamom', 'Cloves', 'Cinnamon',...",NaN
299,299,Garam Masala,https://cooking.nytimes.com/recipes/9650-garam...,"['Coriander', 'Cumin', 'Pepper', 'Corn', 'Card...",Error
310,310,Garam Masala,https://cooking.nytimes.com/recipes/10398-gara...,"['Cardamom', 'Cumin', 'Cloves', 'Pepper', 'Cor...",Error
330,330,Garam Masala,https://cooking.nytimes.com/recipes/2387-garam...,"['Cardamom', 'Cinnamon', 'Cloves', 'Pepper', '...",Error
383,383,Garam Masala,https://cooking.nytimes.com/recipes/6731-garam...,"['Cardamom', 'Cinnamon', 'Cloves', 'Coriander'...",NaN


In [15]:
df['ingredients'] = df['ingredients'].apply(literal_eval) #convert to list type


In [13]:
df[df.ingredients == 'Cardamom' & df.in 'Cinnamon', 'Cloves']

TypeError: 'in <string>' requires string as left operand, not Series

In [19]:
#spreading out all the ingredients, one row for each ingredient for each recipe. 
ingredients = df['ingredients'].explode()

In [17]:
ingredients

0         Flour
0          Milk
0           Egg
0         Sugar
0      Cardamom
         ...   
401    Cardamom
401      Cloves
401      Pepper
401    Cinnamon
401        Salt
Name: ingredients, Length: 6178, dtype: object

In [25]:
#resetting the index so the index is a row so that we can join to the other df 
ingredients_df = ingredients.to_frame().reset_index()

In [31]:
#dropping duplicates because I have salt wayy too many times 
ingredients_df = ingredients_df.drop_duplicates()

In [32]:
ingredients_df

,index,ingredients
0,0,Flour
1,0,Milk
2,0,Egg
3,0,Sugar
4,0,Cardamom
...,...,...
6173,401,Cardamom
6174,401,Cloves
6175,401,Pepper
6176,401,Cinnamon


In [33]:
#rename column of index for the original df to 'index'
df.rename(columns = {'Unnamed: 0':'index'}, inplace = True)


In [37]:
merged_df = df.merge(ingredients_df, on = 'index', how = 'outer')

In [43]:
bar_chart_ingredients = merged_df.value_counts('ingredients_y')

In [44]:
bar_chart_ingredients = bar_chart_ingredients.to_frame()

In [46]:
bar_chart_ingredients.head(50)
#pretty sure gin + pie is just here because it records gin every time there is 'piece of ginger' it records ginger and pie

,count
ingredients_y,
Cardamom,374
Salt,329
Sugar,202
Cinnamon,195
Pepper,178
Butter,173
Gin,163
Cloves,158
Ginger,142


In [47]:
edit_bar_ingredients = bar_chart_ingredients.drop(['Pie', 'Gin', 'Salted butter'])
# I decided to remove pie, gin, and salted butter because every time it lists salted butter it also lists butter
#and every time that it lists piece of ginger it lists pie, gin, and ginger. 

In [49]:
#need to make a bar chart with this 
edit_bar_ingredients.head(50)

,count
ingredients_y,
Cardamom,374
Salt,329
Sugar,202
Cinnamon,195
Pepper,178
Butter,173
Cloves,158
Ginger,142
Egg,131


In [59]:
top_ten = edit_bar_ingredients.drop(['Salt', 'Cardamom']).head(10)

In [61]:
top_ten.to_csv('top_ten.csv')

In [56]:
merged_df[merged_df.ingredients_y== 'Sugar'].head(50)
#finding everything with sugar in it. sweets!!!!

,index,recipe,link,ingredients_x,error,ingredients_y
3,0,Malawax (Cardamom Crepe),https://cooking.nytimes.com/recipes/1023357-ma...,"[Flour, Milk, Egg, Sugar, Cardamom, Salt, Ghee]",NaN,Sugar
8,1,Orange-Cardamom Pancakes,https://cooking.nytimes.com/recipes/1019947-or...,"[Flour, Sugar, Cardamom, Salt, Milk, Egg, Butt...",Error,Sugar
24,2,Brownies With Coffee and Cardamom,https://cooking.nytimes.com/recipes/1024100-br...,"[Salt, Butter, Salted butter, Chocolate, Coffe...",Error,Sugar
28,3,Swedish Cardamom Buns,https://cooking.nytimes.com/recipes/1020437-sw...,"[Milk, Flour, Sugar, Salt, Butter, Salted butt...",Error,Sugar
57,5,Apple Prune Cardamom Cake,https://cooking.nytimes.com/recipes/1019479-ap...,"[Apple, Apple cider, Apple, Salt, Butter, Salt...",Error,Sugar
67,6,Cardamom Butter Crescents,https://cooking.nytimes.com/recipes/1017855-ca...,"[Flour, Cardamom, Salt, Salt, Butter, Salted b...",Error,Sugar
75,7,Red Wine-Pear Cardamom Cake,https://cooking.nytimes.com/recipes/1023646-re...,"[Red wine, Pear, Salt, Butter, Salted butter, ...",Error,Sugar
82,8,Cardamom Cream Cake,https://cooking.nytimes.com/recipes/1017901-ca...,"[Milk, Cream, Sugar, Water, Milk, Cardamom, Su...",Error,Sugar
103,9,"Hazelnut, Pear and Cardamom Tart",https://cooking.nytimes.com/recipes/1020611-ha...,"[Pie crust, Pie, Hazelnut, Nuts, Flour, Cardam...",Error,Sugar
111,10,Cherry Rugelach With Cardamom Sugar,https://cooking.nytimes.com/recipes/1021662-ch...,"[Cheese, Cream, Salt, Butter, Salted butter, S...",Error,Sugar


In [ ]:
#finding indian dishes.
#making a list with curries, chutneys, paneers, etc
indian_food = ['curry', 'pulao', 'pilau', 'biryani','korma', 'paneer', 'masala', 'marsala', 'dal', 'basmati', 'vindaloo','persian']

In [63]:
df.recipe.value_counts()
# inteereesting that there are 5 garam masala recipes !!
#could do a graph on the different ingredients in those 

recipe
Garam Masala                          5
Cranberry Chutney                     2
Mango Lassi                           2
Zhug                                  2
Holiday Stollen                       1
                                     ..
Fuul (Somali-Style Fava Bean Stew)    1
Turmeric Rice With Tomatoes           1
Chicken Braised With Grapes           1
Warm Cabbage and Green Beans          1
Fragrant Brown Rice                   1
Name: count, Length: 395, dtype: int64

In [73]:
recipe_names = df['recipe'].tolist()

In [74]:
recipe_names = [re.sub(r'[^\w\s]', ' ', name.lower()) for name in recipe_names]

In [75]:
all_names = ' '.join(recipe_names)

In [76]:
all_names

'malawax  cardamom crepe  orange cardamom pancakes brownies with coffee and cardamom swedish cardamom buns mackerel with pistachio and cardamom salsa apple prune cardamom cake cardamom butter crescents red wine pear cardamom cake cardamom cream cake hazelnut  pear and cardamom tart cherry rugelach with cardamom sugar strawberry jam bars with cardamom semlor  cardamom cream buns  blackberry crisp with cardamom custard sauce fresh ricotta with rhubarb  ginger and cardamom chocolate cardamom pots de crème gajar halwa  carrot and cardamom pudding  maple cardamom saffron sticky buns mulled cider with cardamom  black pepper and ginger fish koftas in tomato and cardamom sauce grilled cucumbers with tomato cardamom dressing and mozzarella muzaffar seviyan  sweet vermicelli with cardamom  yogurt marinated leg of lamb with cardamom and orange almond cake with cardamom and pistachio leek and cardamom fritters coconut cardamom panna cotta challaw  cardamom and cumin basmati rice  cardamom walnut c

In [77]:
words = all_names.split()

In [78]:
words

['malawax',
 'cardamom',
 'crepe',
 'orange',
 'cardamom',
 'pancakes',
 'brownies',
 'with',
 'coffee',
 'and',
 'cardamom',
 'swedish',
 'cardamom',
 'buns',
 'mackerel',
 'with',
 'pistachio',
 'and',
 'cardamom',
 'salsa',
 'apple',
 'prune',
 'cardamom',
 'cake',
 'cardamom',
 'butter',
 'crescents',
 'red',
 'wine',
 'pear',
 'cardamom',
 'cake',
 'cardamom',
 'cream',
 'cake',
 'hazelnut',
 'pear',
 'and',
 'cardamom',
 'tart',
 'cherry',
 'rugelach',
 'with',
 'cardamom',
 'sugar',
 'strawberry',
 'jam',
 'bars',
 'with',
 'cardamom',
 'semlor',
 'cardamom',
 'cream',
 'buns',
 'blackberry',
 'crisp',
 'with',
 'cardamom',
 'custard',
 'sauce',
 'fresh',
 'ricotta',
 'with',
 'rhubarb',
 'ginger',
 'and',
 'cardamom',
 'chocolate',
 'cardamom',
 'pots',
 'de',
 'crème',
 'gajar',
 'halwa',
 'carrot',
 'and',
 'cardamom',
 'pudding',
 'maple',
 'cardamom',
 'saffron',
 'sticky',
 'buns',
 'mulled',
 'cider',
 'with',
 'cardamom',
 'black',
 'pepper',
 'and',
 'ginger',
 'fish',


In [79]:
word_counts = Counter(words)

In [88]:
df_word_counts = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])

In [89]:
df_word_counts = df_word_counts.sort_values('Count', ascending=False)

In [92]:
df_word_counts.head(10)
#looks like we have some unhelpful values like with, and, s. need to remove those. 


,Word,Count
6,with,109
8,and,85
1,cardamom,48
83,chicken,45
16,cake,28
81,rice,28
159,spiced,28
71,lamb,26
171,s,23
124,curry,16


In [124]:
df_word_counts = (df_word_counts.iloc[2:]).copy()
#good, we got rid of those two lines. Now I can make a graph with the first few of these

In [125]:
df_word_counts.drop(df_word_counts[df_word_counts['Word'] == 's'].index, inplace = True)

In [128]:
ten_df_word_counts = df_word_counts.head(10)


In [134]:
ten_df_word_counts.to_csv('ten_all_recipes.csv')

In [94]:
#how about the recipes with sugar in them? 
sugar_df = merged_df[merged_df.ingredients_y== 'Sugar']

In [95]:
sugar_df

,index,recipe,link,ingredients_x,error,ingredients_y
3,0,Malawax (Cardamom Crepe),https://cooking.nytimes.com/recipes/1023357-ma...,"[Flour, Milk, Egg, Sugar, Cardamom, Salt, Ghee]",NaN,Sugar
8,1,Orange-Cardamom Pancakes,https://cooking.nytimes.com/recipes/1019947-or...,"[Flour, Sugar, Cardamom, Salt, Milk, Egg, Butt...",Error,Sugar
24,2,Brownies With Coffee and Cardamom,https://cooking.nytimes.com/recipes/1024100-br...,"[Salt, Butter, Salted butter, Chocolate, Coffe...",Error,Sugar
28,3,Swedish Cardamom Buns,https://cooking.nytimes.com/recipes/1020437-sw...,"[Milk, Flour, Sugar, Salt, Butter, Salted butt...",Error,Sugar
57,5,Apple Prune Cardamom Cake,https://cooking.nytimes.com/recipes/1019479-ap...,"[Apple, Apple cider, Apple, Salt, Butter, Salt...",Error,Sugar
...,...,...,...,...,...,...
4912,377,Cranberry Chutney,https://cooking.nytimes.com/recipes/81-cranber...,"[Sugar, Apple, Pear, Papaya, Mango, Cardamom, ...",Error,Sugar
4933,379,Srikhand (Yogurt-Based Sweet),https://cooking.nytimes.com/recipes/2388-srikh...,"[Yogurt, Cream, Sour cream, Cardamom, Saffron,...",Error,Sugar
4980,382,Swedish Mulled Wine (Glogg),https://cooking.nytimes.com/recipes/1448-swedi...,"[Red wine, White wine, Lemon, Cloves, Cardamom...",Error,Sugar
5035,387,"Gingerbread Men, Silesian-Style",https://cooking.nytimes.com/recipes/1595-ginge...,"[Honey, Sugar, Butter, Flour, Almond, Lemon, C...",Error,Sugar


In [98]:
sugar_recipe_names = sugar_df['recipe'].tolist()

In [99]:
sugar_recipe_names = [re.sub(r'[^\w\s]', ' ', name.lower()) for name in sugar_recipe_names]

In [100]:
sugar_all_names = ' '.join(sugar_recipe_names)

In [101]:
sugar_words = sugar_all_names.split()

In [104]:
sugar_word_counts = Counter(sugar_words)

In [105]:
sugar_df_word_counts = pd.DataFrame(list(sugar_word_counts.items()), columns=['Word', 'Count'])

In [106]:
sugar_df_word_counts = sugar_df_word_counts.sort_values('Count', ascending=False)

In [116]:
sugar_df_word_counts.head(10)
# we need to delete the with and and from it

,Word,Count
1,cardamom,32
13,cake,28
106,spiced,14
107,pumpkin,12
19,cream,12
44,pudding,10
79,pie,10
65,rice,10
128,caramel,8
18,pear,8


In [110]:
sugar_df_word_counts.drop(sugar_df_word_counts[sugar_df_word_counts['Word'] == 'and'].index, inplace = True)

In [112]:
sugar_df_word_counts.drop(sugar_df_word_counts[sugar_df_word_counts['Word'] == 'with'].index, inplace = True)

In [130]:
ten_sugar_df = sugar_df_word_counts.head(10)

In [133]:
ten_sugar_df.to_csv('top_ten_sugar.csv')

In [18]:
df[df.recipe.str.contains('Cardamom')]

,Unnamed: 0,recipe,link,ingredients,error
0,0,Malawax (Cardamom Crepe),https://cooking.nytimes.com/recipes/1023357-ma...,"['Flour', 'Milk', 'Egg', 'Sugar', 'Cardamom', ...",NaN
1,1,Orange-Cardamom Pancakes,https://cooking.nytimes.com/recipes/1019947-or...,"['Flour', 'Sugar', 'Cardamom', 'Salt', 'Milk',...",Error
2,2,Brownies With Coffee and Cardamom,https://cooking.nytimes.com/recipes/1024100-br...,"['Salt', 'Butter', 'Salted butter', 'Chocolate...",Error
3,3,Swedish Cardamom Buns,https://cooking.nytimes.com/recipes/1020437-sw...,"['Milk', 'Flour', 'Sugar', 'Salt', 'Butter', '...",Error
4,4,Mackerel With Pistachio and Cardamom Salsa,https://cooking.nytimes.com/recipes/1019311-ma...,"['Cardamom', 'Salt', 'Ginger', 'Gin', 'Pie', '...",Error
5,5,Apple Prune Cardamom Cake,https://cooking.nytimes.com/recipes/1019479-ap...,"['Apple', 'Apple cider', 'Apple', 'Salt', 'But...",Error
6,6,Cardamom Butter Crescents,https://cooking.nytimes.com/recipes/1017855-ca...,"['Flour', 'Cardamom', 'Salt', 'Salt', 'Butter'...",Error
7,7,Red Wine-Pear Cardamom Cake,https://cooking.nytimes.com/recipes/1023646-re...,"['Red wine', 'Pear', 'Salt', 'Butter', 'Salted...",Error
8,8,Cardamom Cream Cake,https://cooking.nytimes.com/recipes/1017901-ca...,"['Milk', 'Cream', 'Sugar', 'Water', 'Milk', 'C...",Error
9,9,"Hazelnut, Pear and Cardamom Tart",https://cooking.nytimes.com/recipes/1020611-ha...,"['Pie crust', 'Pie', 'Hazelnut', 'Nuts', 'Flou...",Error
